# ANALYSIS 1: Avocado value, volume, and unit volume

# Avocado Value

In [222]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import json
from sqlalchemy import create_engine

In [223]:
avocado_value_to_load = "Resources/avocado_value.csv"

avocado_value_df = pd.read_csv(avocado_value_to_load)
avocado_value_df.head(30)

,"Avocados: U.S. imports by value ($1,000)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Quarter 1,NaN,NaN,NaN,Quarter 2,NaN,NaN,...,Quarter 3,NaN,NaN,NaN,Quarter 4,NaN,NaN,NaN,Ann,NaN
2,NaN,Fresh¹,Market year*,Jan,Feb,Mar,Q1,Apr,NaN,May,...,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,NaN
3,NaN,NaN,2018,"210,504","182,140","206,933","599,577","223,211",NaN,"179,488",...,"188,226","262,554","190,555","641,335","182,957","178,485","163,051","524,492","2,348,390",NaN
4,NaN,NaN,2017,"207,388","182,421","224,948","614,757","224,357",NaN,"210,834",...,"181,054","257,137","322,733","760,924","220,883","170,644","216,061","607,588","2,639,559",NaN
5,NaN,NaN,2016,"161,251","113,829","121,066","396,145","120,049",NaN,"144,379",...,"171,467","164,853","242,517","578,838","239,007","158,690","175,439","573,137","1,916,917",NaN
6,NaN,NaN,2015,"173,913","124,449","143,180","441,542","127,595",NaN,"131,782",...,"160,925","129,984","135,746","426,654","129,008","130,706","114,402","374,117","1,631,504",NaN
7,NaN,NaN,2014,"149,422","111,138","137,833","398,392","123,645",NaN,"114,155",...,"125,406","110,745","149,557","385,708","135,718","114,327","118,674","368,719","1,526,375",NaN
8,NaN,NaN,2013,"105,584","68,668","104,974","279,225","93,601",NaN,"91,417",...,"66,252","72,171","85,962","224,384","120,382","102,556","100,272","323,210","1,089,341",NaN
9,NaN,NaN,2012,"102,787","72,417","86,286","261,490","82,345",NaN,"63,444",...,"46,183","58,385","51,152","155,720","82,295","76,363","78,649","237,307","860,142",NaN


In [224]:
avocado_value_df.drop([0, 1, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27], inplace=True)

In [225]:
avocado_value_df.head(30)

,"Avocados: U.S. imports by value ($1,000)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
2,NaN,Fresh¹,Market year*,Jan,Feb,Mar,Q1,Apr,NaN,May,...,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,NaN
3,NaN,NaN,2018,"210,504","182,140","206,933","599,577","223,211",NaN,"179,488",...,"188,226","262,554","190,555","641,335","182,957","178,485","163,051","524,492","2,348,390",NaN
4,NaN,NaN,2017,"207,388","182,421","224,948","614,757","224,357",NaN,"210,834",...,"181,054","257,137","322,733","760,924","220,883","170,644","216,061","607,588","2,639,559",NaN
5,NaN,NaN,2016,"161,251","113,829","121,066","396,145","120,049",NaN,"144,379",...,"171,467","164,853","242,517","578,838","239,007","158,690","175,439","573,137","1,916,917",NaN
6,NaN,NaN,2015,"173,913","124,449","143,180","441,542","127,595",NaN,"131,782",...,"160,925","129,984","135,746","426,654","129,008","130,706","114,402","374,117","1,631,504",NaN
7,NaN,NaN,2014,"149,422","111,138","137,833","398,392","123,645",NaN,"114,155",...,"125,406","110,745","149,557","385,708","135,718","114,327","118,674","368,719","1,526,375",NaN
8,NaN,NaN,2013,"105,584","68,668","104,974","279,225","93,601",NaN,"91,417",...,"66,252","72,171","85,962","224,384","120,382","102,556","100,272","323,210","1,089,341",NaN
9,NaN,NaN,2012,"102,787","72,417","86,286","261,490","82,345",NaN,"63,444",...,"46,183","58,385","51,152","155,720","82,295","76,363","78,649","237,307","860,142",NaN
10,NaN,NaN,2011,"101,587","75,227","97,560","274,375","86,213",NaN,"66,860",...,"36,879","69,722","92,924","199,524","77,476","70,680","75,893","224,049","912,999",NaN
11,NaN,NaN,2010,"66,200","48,118","65,695","180,013","51,758",NaN,"41,807",...,"29,989","24,075","35,773","89,836","46,616","56,786","72,570","175,971","574,698",NaN


In [226]:
avocado_value_df.drop(['Avocados: U.S. imports by value ($1,000)','Unnamed: 1', 'Unnamed: 8', 'Unnamed: 10', 'Unnamed: 22'], axis=1, inplace=True)

In [227]:
avocado_value_df.head(20)

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 9,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
2,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
3,2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226","262,554","190,555","641,335","182,957","178,485","163,051","524,492","2,348,390"
4,2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054","257,137","322,733","760,924","220,883","170,644","216,061","607,588","2,639,559"
5,2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467","164,853","242,517","578,838","239,007","158,690","175,439","573,137","1,916,917"
6,2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925","129,984","135,746","426,654","129,008","130,706","114,402","374,117","1,631,504"
7,2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406","110,745","149,557","385,708","135,718","114,327","118,674","368,719","1,526,375"
8,2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252","72,171","85,962","224,384","120,382","102,556","100,272","323,210","1,089,341"
9,2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183","58,385","51,152","155,720","82,295","76,363","78,649","237,307","860,142"
10,2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879","69,722","92,924","199,524","77,476","70,680","75,893","224,049","912,999"
11,2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989","24,075","35,773","89,836","46,616","56,786","72,570","175,971","574,698"


In [228]:
# Create a new variable called 'header' from the first row of the dataset
header = avocado_value_df.iloc[0]
# Take the data less the header row
avocado_value_df = avocado_value_df[1:] 
# Rename the dataframe's column values with the header variable
avocado_value_df.rename(columns = header, inplace=True)

In [229]:
avocado_value_df

,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
3,2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226","262,554","190,555","641,335","182,957","178,485","163,051","524,492","2,348,390"
4,2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054","257,137","322,733","760,924","220,883","170,644","216,061","607,588","2,639,559"
5,2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467","164,853","242,517","578,838","239,007","158,690","175,439","573,137","1,916,917"
6,2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925","129,984","135,746","426,654","129,008","130,706","114,402","374,117","1,631,504"
7,2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406","110,745","149,557","385,708","135,718","114,327","118,674","368,719","1,526,375"
8,2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252","72,171","85,962","224,384","120,382","102,556","100,272","323,210","1,089,341"
9,2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183","58,385","51,152","155,720","82,295","76,363","78,649","237,307","860,142"
10,2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879","69,722","92,924","199,524","77,476","70,680","75,893","224,049","912,999"
11,2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989","24,075","35,773","89,836","46,616","56,786","72,570","175,971","574,698"
12,2009,"69,880","52,776","66,444","189,100","69,952","54,434","60,462","184,849","48,126","65,198","61,818","175,141","61,124","49,005","51,957","162,087","711,177"


In [230]:
avocado_value_df.set_index('Market year*')

,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
Market year*,,,,,,,,,,,,,,,,,
2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226","262,554","190,555","641,335","182,957","178,485","163,051","524,492","2,348,390"
2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054","257,137","322,733","760,924","220,883","170,644","216,061","607,588","2,639,559"
2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467","164,853","242,517","578,838","239,007","158,690","175,439","573,137","1,916,917"
2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925","129,984","135,746","426,654","129,008","130,706","114,402","374,117","1,631,504"
2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406","110,745","149,557","385,708","135,718","114,327","118,674","368,719","1,526,375"
2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252","72,171","85,962","224,384","120,382","102,556","100,272","323,210","1,089,341"
2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183","58,385","51,152","155,720","82,295","76,363","78,649","237,307","860,142"
2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879","69,722","92,924","199,524","77,476","70,680","75,893","224,049","912,999"
2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989","24,075","35,773","89,836","46,616","56,786","72,570","175,971","574,698"


## Avocado Volume

In [231]:
avocado_volume_to_load = "Resources/avocado_volume.csv"

avocado_volume_df = pd.read_csv(avocado_volume_to_load)
avocado_volume_df.head(35)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Avocados: U.S. imports by volume (1,000 pounds)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Quarter 1,NaN,NaN,NaN,NaN,Quarter 2,NaN,...,Quarter 3,NaN,NaN,NaN,Quarter 4,NaN,NaN,NaN,Ann,NaN
4,Fresh¹,NaN,Market year*,Jan,NaN,Feb,Mar,Q1,Apr,May,...,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,NaN
5,NaN,NaN,2018,"216,705",NaN,"184,358","202,821","603,884","197,119","171,803",...,"177,086","198,703","177,494","553,283","198,663","176,578","207,942","583,183","2,288,645",NaN
6,NaN,NaN,2017,"214,330",NaN,"157,957","171,411","543,698","155,963","143,676",...,"140,090","140,622","158,130","438,841","176,368","169,804","191,073","537,244","1,984,597",NaN
7,NaN,NaN,2016,"228,436",NaN,"170,883","183,434","582,753","178,464","141,989",...,"129,448","131,575","155,449","416,471","151,707","149,558","184,931","486,196","1,895,108",NaN
8,NaN,NaN,2015,"193,351",NaN,"138,164","149,668","481,183","129,485","140,010",...,"167,218","158,227","173,893","499,338","178,050","173,883","166,455","518,388","1,912,252",NaN
9,NaN,NaN,2014,"173,353",NaN,"129,254","144,845","447,452","117,249","103,882",...,"119,387","111,120","141,528","372,036","158,640","139,024","144,572","442,236","1,607,530",NaN


In [232]:
avocado_volume_df.drop([0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], inplace=True)

In [233]:
avocado_volume_df.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
4,Fresh¹,NaN,Market year*,Jan,NaN,Feb,Mar,Q1,Apr,May,...,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total,NaN
5,NaN,NaN,2018,"216,705",NaN,"184,358","202,821","603,884","197,119","171,803",...,"177,086","198,703","177,494","553,283","198,663","176,578","207,942","583,183","2,288,645",NaN
6,NaN,NaN,2017,"214,330",NaN,"157,957","171,411","543,698","155,963","143,676",...,"140,090","140,622","158,130","438,841","176,368","169,804","191,073","537,244","1,984,597",NaN
7,NaN,NaN,2016,"228,436",NaN,"170,883","183,434","582,753","178,464","141,989",...,"129,448","131,575","155,449","416,471","151,707","149,558","184,931","486,196","1,895,108",NaN
8,NaN,NaN,2015,"193,351",NaN,"138,164","149,668","481,183","129,485","140,010",...,"167,218","158,227","173,893","499,338","178,050","173,883","166,455","518,388","1,912,252",NaN
9,NaN,NaN,2014,"173,353",NaN,"129,254","144,845","447,452","117,249","103,882",...,"119,387","111,120","141,528","372,036","158,640","139,024","144,572","442,236","1,607,530",NaN
10,NaN,NaN,2013,"167,603",NaN,"111,168","129,819","408,590","102,694","97,904",...,"63,872","64,158","73,871","201,901","123,323","125,620","127,530","376,473","1,260,588",NaN
11,NaN,NaN,2012,"134,950",NaN,"97,445","101,860","334,255","87,487","66,922",...,"53,227","69,665","69,269","192,161","114,218","119,587","128,016","361,821","1,107,969",NaN
12,NaN,NaN,2011,"119,050",NaN,"82,960","80,025","282,036","66,248","50,263",...,"23,534","57,441","92,352","173,327","101,182","94,418","106,837","302,438","915,592",NaN
13,NaN,NaN,2010,"110,978",NaN,"81,091","89,568","281,637","60,707","46,366",...,"32,322","26,394","38,718","97,434","61,651","76,736","98,195","236,582","760,616",NaN


In [234]:
avocado_volume_df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 4','Unnamed: 10', 'Unnamed: 22'], axis=1, inplace=True)

In [235]:
avocado_volume_df.head(20)

,Unnamed: 2,Unnamed: 3,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
4,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
5,2018,"216,705","184,358","202,821","603,884","197,119","171,803","179,373","548,295","177,086","198,703","177,494","553,283","198,663","176,578","207,942","583,183","2,288,645"
6,2017,"214,330","157,957","171,411","543,698","155,963","143,676","165,175","464,814","140,090","140,622","158,130","438,841","176,368","169,804","191,073","537,244","1,984,597"
7,2016,"228,436","170,883","183,434","582,753","178,464","141,989","89,235","409,688","129,448","131,575","155,449","416,471","151,707","149,558","184,931","486,196","1,895,108"
8,2015,"193,351","138,164","149,668","481,183","129,485","140,010","143,848","413,343","167,218","158,227","173,893","499,338","178,050","173,883","166,455","518,388","1,912,252"
9,2014,"173,353","129,254","144,845","447,452","117,249","103,882","124,675","345,806","119,387","111,120","141,528","372,036","158,640","139,024","144,572","442,236","1,607,530"
10,2013,"167,603","111,168","129,819","408,590","102,694","97,904","73,026","273,623","63,872","64,158","73,871","201,901","123,323","125,620","127,530","376,473","1,260,588"
11,2012,"134,950","97,445","101,860","334,255","87,487","66,922","65,323","219,732","53,227","69,665","69,269","192,161","114,218","119,587","128,016","361,821","1,107,969"
12,2011,"119,050","82,960","80,025","282,036","66,248","50,263","41,281","157,792","23,534","57,441","92,352","173,327","101,182","94,418","106,837","302,438","915,592"
13,2010,"110,978","81,091","89,568","281,637","60,707","46,366","37,889","144,962","32,322","26,394","38,718","97,434","61,651","76,736","98,195","236,582","760,616"


In [236]:
# Create a new variable called 'header' from the first row of the dataset
header = avocado_volume_df.iloc[0]
# Take the data less the header row
avocado_volume_df = avocado_volume_df[1:] 
# Rename the dataframe's column values with the header variable
avocado_volume_df.rename(columns = header, inplace=True)

In [237]:
avocado_volume_df

,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
5,2018,"216,705","184,358","202,821","603,884","197,119","171,803","179,373","548,295","177,086","198,703","177,494","553,283","198,663","176,578","207,942","583,183","2,288,645"
6,2017,"214,330","157,957","171,411","543,698","155,963","143,676","165,175","464,814","140,090","140,622","158,130","438,841","176,368","169,804","191,073","537,244","1,984,597"
7,2016,"228,436","170,883","183,434","582,753","178,464","141,989","89,235","409,688","129,448","131,575","155,449","416,471","151,707","149,558","184,931","486,196","1,895,108"
8,2015,"193,351","138,164","149,668","481,183","129,485","140,010","143,848","413,343","167,218","158,227","173,893","499,338","178,050","173,883","166,455","518,388","1,912,252"
9,2014,"173,353","129,254","144,845","447,452","117,249","103,882","124,675","345,806","119,387","111,120","141,528","372,036","158,640","139,024","144,572","442,236","1,607,530"
10,2013,"167,603","111,168","129,819","408,590","102,694","97,904","73,026","273,623","63,872","64,158","73,871","201,901","123,323","125,620","127,530","376,473","1,260,588"
11,2012,"134,950","97,445","101,860","334,255","87,487","66,922","65,323","219,732","53,227","69,665","69,269","192,161","114,218","119,587","128,016","361,821","1,107,969"
12,2011,"119,050","82,960","80,025","282,036","66,248","50,263","41,281","157,792","23,534","57,441","92,352","173,327","101,182","94,418","106,837","302,438","915,592"
13,2010,"110,978","81,091","89,568","281,637","60,707","46,366","37,889","144,962","32,322","26,394","38,718","97,434","61,651","76,736","98,195","236,582","760,616"
14,2009,"102,877","77,481","86,440","266,798","74,690","55,237","54,946","184,873","46,103","82,939","90,122","219,164","97,694","88,738","94,543","280,975","951,810"


In [238]:
avocado_volume_df.set_index('Market year*')

,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Total
Market year*,,,,,,,,,,,,,,,,,
2018,"216,705","184,358","202,821","603,884","197,119","171,803","179,373","548,295","177,086","198,703","177,494","553,283","198,663","176,578","207,942","583,183","2,288,645"
2017,"214,330","157,957","171,411","543,698","155,963","143,676","165,175","464,814","140,090","140,622","158,130","438,841","176,368","169,804","191,073","537,244","1,984,597"
2016,"228,436","170,883","183,434","582,753","178,464","141,989","89,235","409,688","129,448","131,575","155,449","416,471","151,707","149,558","184,931","486,196","1,895,108"
2015,"193,351","138,164","149,668","481,183","129,485","140,010","143,848","413,343","167,218","158,227","173,893","499,338","178,050","173,883","166,455","518,388","1,912,252"
2014,"173,353","129,254","144,845","447,452","117,249","103,882","124,675","345,806","119,387","111,120","141,528","372,036","158,640","139,024","144,572","442,236","1,607,530"
2013,"167,603","111,168","129,819","408,590","102,694","97,904","73,026","273,623","63,872","64,158","73,871","201,901","123,323","125,620","127,530","376,473","1,260,588"
2012,"134,950","97,445","101,860","334,255","87,487","66,922","65,323","219,732","53,227","69,665","69,269","192,161","114,218","119,587","128,016","361,821","1,107,969"
2011,"119,050","82,960","80,025","282,036","66,248","50,263","41,281","157,792","23,534","57,441","92,352","173,327","101,182","94,418","106,837","302,438","915,592"
2010,"110,978","81,091","89,568","281,637","60,707","46,366","37,889","144,962","32,322","26,394","38,718","97,434","61,651","76,736","98,195","236,582","760,616"


## Avocado Unit Value

In [239]:
avocado_unitvalue_to_load = "Resources/avocado_unitvalue.csv"

avocado_unitvalue_df = pd.read_csv(avocado_unitvalue_to_load)
avocado_unitvalue_df.head(30)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Avocados: U.S. imports unit value ($ per pound),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Quarter 1,NaN,NaN,NaN,Quarter 2,NaN,NaN,...,NaN,Quarter 3,NaN,NaN,NaN,Quarter 4,NaN,NaN,NaN,NaN
4,Fresh¹,NaN,Market year*,Jan,Feb,Mar,Q1,Apr,May,NaN,...,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
5,NaN,NaN,2018,0.97,0.99,1.02,0.99,1.13,1.04,NaN,...,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
6,NaN,NaN,2017,0.97,1.15,1.31,1.13,1.44,1.47,NaN,...,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
7,NaN,NaN,2016,0.71,0.67,0.66,0.68,0.67,1.02,NaN,...,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
8,NaN,NaN,2015,0.90,0.90,0.96,0.92,0.99,0.94,NaN,...,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
9,NaN,NaN,2014,0.86,0.86,0.95,0.89,1.05,1.10,NaN,...,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95


In [240]:
avocado_unitvalue_df.drop([0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30], inplace=True)

In [241]:
avocado_unitvalue_df.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
4,Fresh¹,NaN,Market year*,Jan,Feb,Mar,Q1,Apr,May,NaN,...,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
5,NaN,NaN,2018,0.97,0.99,1.02,0.99,1.13,1.04,NaN,...,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
6,NaN,NaN,2017,0.97,1.15,1.31,1.13,1.44,1.47,NaN,...,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
7,NaN,NaN,2016,0.71,0.67,0.66,0.68,0.67,1.02,NaN,...,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
8,NaN,NaN,2015,0.90,0.90,0.96,0.92,0.99,0.94,NaN,...,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
9,NaN,NaN,2014,0.86,0.86,0.95,0.89,1.05,1.10,NaN,...,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
10,NaN,NaN,2013,0.63,0.62,0.81,0.68,0.91,0.93,NaN,...,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
11,NaN,NaN,2012,0.76,0.74,0.85,0.78,0.94,0.95,NaN,...,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
12,NaN,NaN,2011,0.85,0.91,1.22,0.97,1.30,1.33,NaN,...,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
13,NaN,NaN,2010,0.60,0.59,0.73,0.64,0.85,0.90,NaN,...,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76


In [242]:
avocado_unitvalue_df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 9', 'Unnamed: 10'], axis=1, inplace=True)

In [243]:
avocado_unitvalue_df.head(20)

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
4,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
5,2018,0.97,0.99,1.02,0.99,1.13,1.04,1.01,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
6,2017,0.97,1.15,1.31,1.13,1.44,1.47,1.34,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
7,2016,0.71,0.67,0.66,0.68,0.67,1.02,1.17,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
8,2015,0.90,0.90,0.96,0.92,0.99,0.94,0.90,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
9,2014,0.86,0.86,0.95,0.89,1.05,1.10,1.09,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
10,2013,0.63,0.62,0.81,0.68,0.91,0.93,1.06,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
11,2012,0.76,0.74,0.85,0.78,0.94,0.95,0.92,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
12,2011,0.85,0.91,1.22,0.97,1.30,1.33,1.50,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
13,2010,0.60,0.59,0.73,0.64,0.85,0.90,0.93,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76


In [244]:
# Create a new variable called 'header' from the first row of the dataset
header = avocado_unitvalue_df.iloc[0]
# Take the data less the header row
avocado_unitvalue_df = avocado_unitvalue_df[1:] 
# Rename the dataframe's column values with the header variable
avocado_unitvalue_df.rename(columns = header, inplace=True)

In [245]:
avocado_unitvalue_df

,Market year*,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
5,2018,0.97,0.99,1.02,0.99,1.13,1.04,1.01,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
6,2017,0.97,1.15,1.31,1.13,1.44,1.47,1.34,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
7,2016,0.71,0.67,0.66,0.68,0.67,1.02,1.17,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
8,2015,0.90,0.90,0.96,0.92,0.99,0.94,0.90,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
9,2014,0.86,0.86,0.95,0.89,1.05,1.10,1.09,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
10,2013,0.63,0.62,0.81,0.68,0.91,0.93,1.06,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
11,2012,0.76,0.74,0.85,0.78,0.94,0.95,0.92,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
12,2011,0.85,0.91,1.22,0.97,1.30,1.33,1.50,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
13,2010,0.60,0.59,0.73,0.64,0.85,0.90,0.93,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76
14,2009,0.68,0.68,0.77,0.71,0.94,0.99,1.10,1.00,1.04,0.79,0.69,0.80,0.63,0.55,0.55,0.58,0.75


In [246]:
avocado_unitvalue_df.set_index('Market year*')

,Jan,Feb,Mar,Q1,Apr,May,Jun,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
Market year*,,,,,,,,,,,,,,,,,
2018,0.97,0.99,1.02,0.99,1.13,1.04,1.01,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
2017,0.97,1.15,1.31,1.13,1.44,1.47,1.34,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
2016,0.71,0.67,0.66,0.68,0.67,1.02,1.17,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
2015,0.90,0.90,0.96,0.92,0.99,0.94,0.90,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
2014,0.86,0.86,0.95,0.89,1.05,1.10,1.09,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
2013,0.63,0.62,0.81,0.68,0.91,0.93,1.06,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
2012,0.76,0.74,0.85,0.78,0.94,0.95,0.92,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
2011,0.85,0.91,1.22,0.97,1.30,1.33,1.50,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
2010,0.60,0.59,0.73,0.64,0.85,0.90,0.93,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76


## Merge 3 Tables

In [247]:
from functools import reduce

# Compile the list of dataframes 
all_avocado_df = [avocado_value_df, avocado_volume_df, avocado_unitvalue_df]

In [248]:
avocado_merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['Market year*'],
                                            how='outer'), all_avocado_df)

In [249]:
avocado_merged_df

,Market year*,Jan_x,Feb_x,Mar_x,Q1_x,Apr_x,May_x,Jun_x,Q2_x,Jul_x,...,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
0,2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226",...,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
1,2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054",...,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
2,2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467",...,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
3,2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925",...,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
4,2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406",...,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
5,2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252",...,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
6,2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183",...,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
7,2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879",...,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
8,2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989",...,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76
9,2009,"69,880","52,776","66,444","189,100","69,952","54,434","60,462","184,849","48,126",...,1.00,1.04,0.79,0.69,0.80,0.63,0.55,0.55,0.58,0.75


In [250]:
avocado_merged_df.all()

Market year*    True
Jan_x           True
Feb_x           True
Mar_x           True
Q1_x            True
Apr_x           True
May_x           True
Jun_x           True
Q2_x            True
Jul_x           True
Aug_x           True
Sep_x           True
Q3_x            True
Oct_x           True
Nov_x           True
Dec_x           True
Q4_x            True
Total_x         True
Jan_y           True
Feb_y           True
Mar_y           True
Q1_y            True
Apr_y           True
May_y           True
Jun_y           True
Q2_y            True
Jul_y           True
Aug_y           True
Sep_y           True
Q3_y            True
Oct_y           True
Nov_y           True
Dec_y           True
Q4_y            True
Total_y         True
Jan             True
Feb             True
Mar             True
Q1              True
Apr             True
May             True
Jun             True
Q2              True
Jul             True
Aug             True
Sep             True
Q3              True
Oct          

In [251]:
#Change the column names
avocado_merged_df = reduce(lambda  left,right: pd.merge(left,right,on=['Market year*'],
                                            how='outer', suffixes=('_value','_volume')), all_avocado_df)

In [252]:
avocado_merged_df

,Market year*,Jan_value,Feb_value,Mar_value,Q1_value,Apr_value,May_value,Jun_value,Q2_value,Jul_value,...,Q2,Jul,Aug,Sep,Q3,Oct,Nov,Dec,Q4,Avg
0,2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226",...,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
1,2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054",...,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
2,2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467",...,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
3,2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925",...,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
4,2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406",...,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
5,2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252",...,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
6,2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183",...,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
7,2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879",...,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
8,2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989",...,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76
9,2009,"69,880","52,776","66,444","189,100","69,952","54,434","60,462","184,849","48,126",...,1.00,1.04,0.79,0.69,0.80,0.63,0.55,0.55,0.58,0.75


In [253]:
avocado_merged_df.rename(columns={'Jan': 'Jan_unitvalue', 'Feb': 'Feb_unitvalue', 'Mar': 'Mar_unitvalue', 'Q1': 'Q1_unitvalue',\
                                  'Apr': 'Apr_unitvalue', 'May': 'May_unitvalue', 'Jun': 'Jun_unitvalue', 'Q2': 'Q2_unitvalue',\
                                  'Jul': 'Jul_unitvalue', 'Aug': 'Aug_unitvalue', 'Sep': 'Sep_unitvalue', 'Q3': 'Q3_unitvalue',\
                                  'Oct': 'Oct_unitvalue', 'Nov': 'Nov_unitvalue', 'Dec': 'Dec_unitvalue', 'Q4': 'Q4_unitvalue',\
                                  'Avg': 'Avg_unitvalue'}, inplace=True)

In [254]:
avocado_merged_df

,Market year*,Jan_value,Feb_value,Mar_value,Q1_value,Apr_value,May_value,Jun_value,Q2_value,Jul_value,...,Q2_unitvalue,Jul_unitvalue,Aug_unitvalue,Sep_unitvalue,Q3_unitvalue,Oct_unitvalue,Nov_unitvalue,Dec_unitvalue,Q4_unitvalue,Avg_unitvalue
0,2018,"210,504","182,140","206,933","599,577","223,211","179,488","180,286","582,985","188,226",...,1.06,1.06,1.32,1.07,1.16,0.92,1.01,0.78,0.90,1.03
1,2017,"207,388","182,421","224,948","614,757","224,357","210,834","221,098","656,289","181,054",...,1.41,1.29,1.83,2.04,1.73,1.25,1.00,1.13,1.13,1.33
2,2016,"161,251","113,829","121,066","396,145","120,049","144,379","104,369","368,797","171,467",...,0.90,1.32,1.25,1.56,1.39,1.58,1.06,0.95,1.18,1.01
3,2015,"173,913","124,449","143,180","441,542","127,595","131,782","129,814","389,191","160,925",...,0.94,0.96,0.82,0.78,0.85,0.72,0.75,0.69,0.72,0.85
4,2014,"149,422","111,138","137,833","398,392","123,645","114,155","135,756","373,555","125,406",...,1.08,1.05,1.00,1.06,1.04,0.86,0.82,0.82,0.83,0.95
5,2013,"105,584","68,668","104,974","279,225","93,601","91,417","77,504","262,522","66,252",...,0.96,1.04,1.12,1.16,1.11,0.98,0.82,0.79,0.86,0.86
6,2012,"102,787","72,417","86,286","261,490","82,345","63,444","59,836","205,625","46,183",...,0.94,0.87,0.84,0.74,0.81,0.72,0.64,0.61,0.66,0.78
7,2011,"101,587","75,227","97,560","274,375","86,213","66,860","61,978","215,051","36,879",...,1.36,1.57,1.21,1.01,1.15,0.77,0.75,0.71,0.74,1.00
8,2010,"66,200","48,118","65,695","180,013","51,758","41,807","35,313","128,878","29,989",...,0.89,0.93,0.91,0.92,0.92,0.76,0.74,0.74,0.74,0.76
9,2009,"69,880","52,776","66,444","189,100","69,952","54,434","60,462","184,849","48,126",...,1.00,1.04,0.79,0.69,0.80,0.63,0.55,0.55,0.58,0.75


In [255]:
# Shape from wide to long with melt function in pandas
df2=pd.melt(avocado_merged_df,id_vars=['Market year*'], var_name='metrics', value_name='values')
df2.head(200)

,Market year*,metrics,values
0,2018,Jan_value,"210,504"
1,2017,Jan_value,"207,388"
2,2016,Jan_value,"161,251"
3,2015,Jan_value,"173,913"
4,2014,Jan_value,"149,422"
5,2013,Jan_value,"105,584"
6,2012,Jan_value,"102,787"
7,2011,Jan_value,"101,587"
8,2010,Jan_value,"66,200"
9,2009,Jan_value,"69,880"


# ANALYSIS 2: 

In [256]:
#read the csv files
retail_csv = "./Resources/avocado.csv"
retail_info = pd.read_csv(retail_csv)

volume_csv_2016 = "./Resources/Volume Data - Hass Avocado Board2016.csv"
volumes_2016 = pd.read_csv(volume_csv_2016)

volume_csv_2017 = "./Resources/Volume Data - Hass Avocado Board2017.csv"
volumes_2017 = pd.read_csv(volume_csv_2017)

volume_csv_2018 = "./Resources/Volume Data - Hass Avocado Board2018.csv"
volumes_2018 = pd.read_csv(volume_csv_2018)

In [257]:
trade_csv ="./Resources/TradeTables.csv"
trade_info = pd.read_excel(trade_csv, skiprows=None)

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbfTextb'

In [258]:
#convert retail data csv to dataframe
retail_df = pd.DataFrame(retail_info)
retail_df.head()

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,12/27/15,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,12/20/15,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,12/13/15,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,12/6/15,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,11/29/15,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [259]:
#convert 2016 U.S. avocodo import data from csv to dataframe
volumes_2016 = pd.read_csv(volume_csv_2016)
volumes_2016.head()

,Week,≈ =,Total Volume,California,Chile,Mexico,Peru
0,"Jan 03, 2016",Actual,"35,186,694","237,957","1,447,516","33,501,221",0
1,"Jan 10, 2016",Actual,"50,608,874","125,175","336,744","50,146,955",0
2,"Jan 17, 2016",Actual,"53,312,157","589,231","1,592,406","51,130,520",0
3,"Jan 24, 2016",Actual,"61,164,738","1,885,780","1,127,833","58,151,125",0
4,"Jan 31, 2016",Actual,"55,185,176","2,541,711","1,262,561","51,380,904",0


In [260]:
#convert 2017 U.S. avocodo import data from csv to dataframe
volumes_2017 = pd.read_csv(volume_csv_2017)
volumes_2017.head()

,Week,≈ =,Total Volume,California,Chile,Mexico,Peru
0,"Jan 01, 2017",Actual,"35,100,521","49,737","1,023,459","34,027,325",0
1,"Jan 08, 2017",Actual,"40,920,289","157,060","1,526,467","39,236,762",0
2,"Jan 15, 2017",Actual,"53,723,606","215,612","1,468,592","52,039,402",0
3,"Jan 22, 2017",Actual,"55,606,793","827,113","1,445,898","53,333,782",0
4,"Jan 29, 2017",Actual,"44,287,021","1,027,998","1,111,063","42,147,960",0


In [261]:
#convert 2018 U.S. avocodo import data from csv to dataframe
volumes_2018 = pd.read_csv(volume_csv_2018)
volumes_2018.head()

,Week,≈ =,Total Volume,California,Chile,Mexico,Peru
0,"Jan 07, 2018",Actual,"45,670,156","899,349","519,433","44,251,374",0
1,"Jan 14, 2018",Actual,"52,758,905","830,821","449,003","51,479,081",0
2,"Jan 21, 2018",Actual,"54,698,266","2,913,663","445,127","51,339,476",0
3,"Jan 28, 2018",Actual,"45,299,201","2,698,356","95,491","42,505,354",0
4,"Feb 04, 2018",Actual,"47,371,481","2,635,471",0,"44,736,010",0


In [262]:
#combine import volumes dataframes 
importvolumes_df = pd.concat([volumes_2016, volumes_2017, volumes_2018], ignore_index=True)
importvolumes_df.head()
    

,Week,≈ =,Total Volume,California,Chile,Mexico,Peru
0,"Jan 03, 2016",Actual,"35,186,694","237,957","1,447,516","33,501,221",0
1,"Jan 10, 2016",Actual,"50,608,874","125,175","336,744","50,146,955",0
2,"Jan 17, 2016",Actual,"53,312,157","589,231","1,592,406","51,130,520",0
3,"Jan 24, 2016",Actual,"61,164,738","1,885,780","1,127,833","58,151,125",0
4,"Jan 31, 2016",Actual,"55,185,176","2,541,711","1,262,561","51,380,904",0


In [263]:
totalimportvolumes_df = importvolumes_df[['Week', 'Total Volume']].copy()
totalimportvolumes_df.head()

,Week,Total Volume
0,"Jan 03, 2016","35,186,694"
1,"Jan 10, 2016","50,608,874"
2,"Jan 17, 2016","53,312,157"
3,"Jan 24, 2016","61,164,738"
4,"Jan 31, 2016","55,185,176"


# ANALYSIS 3:

In [264]:
import datetime as dt

In [265]:
csv_file = "./Resources/Avocado.csv"
avocado_df = pd.read_csv(csv_file)
avocado_df.head()


,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,12/27/15,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,12/20/15,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,12/13/15,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,12/6/15,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,11/29/15,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [266]:
avocado_df.dtypes

Unnamed: 0        int64
Date             object
AveragePrice    float64
Total Volume    float64
4046            float64
4225            float64
4770            float64
Total Bags      float64
Small Bags      float64
Large Bags      float64
XLarge Bags     float64
type             object
year              int64
region           object
dtype: object

In [267]:
avocado_df['Date']=pd.to_datetime(avocado_df["Date"], format= "%m/%d/%y")
print(avocado_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 14 columns):
Unnamed: 0      18249 non-null int64
Date            18249 non-null datetime64[ns]
AveragePrice    18249 non-null float64
Total Volume    18249 non-null float64
4046            18249 non-null float64
4225            18249 non-null float64
4770            18249 non-null float64
Total Bags      18249 non-null float64
Small Bags      18249 non-null float64
Large Bags      18249 non-null float64
XLarge Bags     18249 non-null float64
type            18249 non-null object
year            18249 non-null int64
region          18249 non-null object
dtypes: datetime64[ns](1), float64(9), int64(2), object(2)
memory usage: 1.9+ MB
None


In [268]:
avocado_df['Date'] = pd.to_datetime(avocado_df["Date"].dt.strftime('%Y-%m'))

In [269]:
avocado_df["Month"]= avocado_df["Date"].dt.month_name()

In [270]:
avocado_df

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,Month
0,0,2015-12-01,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.00,conventional,2015,Albany,December
1,1,2015-12-01,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.00,conventional,2015,Albany,December
2,2,2015-12-01,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.00,conventional,2015,Albany,December
3,3,2015-12-01,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.00,conventional,2015,Albany,December
4,4,2015-11-01,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.00,conventional,2015,Albany,November
5,5,2015-11-01,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.00,conventional,2015,Albany,November
6,6,2015-11-01,0.99,83453.76,1368.92,73672.72,93.26,8318.86,8196.81,122.05,0.00,conventional,2015,Albany,November
7,7,2015-11-01,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.00,conventional,2015,Albany,November
8,8,2015-11-01,1.02,99811.42,1022.15,87315.57,85.34,11388.36,11104.53,283.83,0.00,conventional,2015,Albany,November
9,9,2015-10-01,1.07,74338.76,842.40,64757.44,113.00,8625.92,8061.47,564.45,0.00,conventional,2015,Albany,October


In [271]:
del avocado_df["4046"]

In [272]:
del avocado_df["4225"]

In [273]:
del avocado_df["4770"]

In [274]:
del avocado_df["Unnamed: 0"]

In [275]:
del avocado_df["Date"]

In [276]:
avocado_df.head()

,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,Month
0,1.33,64236.62,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,December
1,1.35,54876.98,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,December
2,0.93,118220.22,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,December
3,1.08,78992.15,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,December
4,1.28,51039.60,6183.95,5986.26,197.69,0.0,conventional,2015,Albany,November


In [277]:
avocado= avocado_df[["year", "Month", "region", "type", "AveragePrice", "Total Volume", "Total Bags", "Small Bags", "Large Bags", "XLarge Bags" ]]

In [278]:
avocado.head()

,year,Month,region,type,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags
0,2015,December,Albany,conventional,1.33,64236.62,8696.87,8603.62,93.25,0.0
1,2015,December,Albany,conventional,1.35,54876.98,9505.56,9408.07,97.49,0.0
2,2015,December,Albany,conventional,0.93,118220.22,8145.35,8042.21,103.14,0.0
3,2015,December,Albany,conventional,1.08,78992.15,5811.16,5677.40,133.76,0.0
4,2015,November,Albany,conventional,1.28,51039.60,6183.95,5986.26,197.69,0.0


In [279]:
avocado_clean= avocado.rename(columns={"Date":"Month","year":"Year","region":"Region", "type":"Type"})

In [280]:
avocado_clean.head()

,Year,Month,Region,Type,AveragePrice,Total Volume,Total Bags,Small Bags,Large Bags,XLarge Bags
0,2015,December,Albany,conventional,1.33,64236.62,8696.87,8603.62,93.25,0.0
1,2015,December,Albany,conventional,1.35,54876.98,9505.56,9408.07,97.49,0.0
2,2015,December,Albany,conventional,0.93,118220.22,8145.35,8042.21,103.14,0.0
3,2015,December,Albany,conventional,1.08,78992.15,5811.16,5677.40,133.76,0.0
4,2015,November,Albany,conventional,1.28,51039.60,6183.95,5986.26,197.69,0.0


# ANALYSIS 4:

## Cleaned Export CSV:

In [281]:
csv_file = "./Resources/avocado_merged_exp.csv"
avocado_export_merged = pd.read_csv(csv_file)
avocado_export_merged.head()

,Unnamed: 0,Market year*,metrics,values
0,0,2018,Jan_exp_value,"2,447"
1,1,2017,Jan_exp_value,"1,661"
2,2,2016,Jan_exp_value,788
3,3,2015,Jan_exp_value,610
4,4,2014,Jan_exp_value,491


# LOAD DATA:
## Connect to Local DB

In [284]:
!pip install mysqlclient

    100% |████████████████████████████████| 92kB 2.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for mysqlclient ... done
  Stored in directory: /Users/morganpowell/Library/Caches/pip/wheels/30/91/e0/2ee952bce05b1247807405c6710c6130e49468a5240ae27134
Successfully built mysqlclient


In [300]:
rds_connection_string = "<username>:<password>@127.0.0.1/ETL_Project"
engine = create_engine(f'mysql://{rds_connection_string}', encoding='utf-8')

## Check for Tables

In [301]:
engine.table_names()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

### Use pandas to load csv converted DataFrame into database

In [302]:
df2.to_sql(name='Avocado_Export_Data1', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [303]:
totalimportvolumes_df.to_sql(name='Avocado_Import_Data', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [304]:
avocado_clean.to_sql(name='Avocados_by_Region', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [305]:
avocado_export_merged.to_sql(name='Avocado_Export_Data2', con=engine, if_exists='append', index=False)

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

## Confirm tables:

In [306]:
pd.read_sql_query('select * from Avocado_Export_Data1', con=engine).head()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [307]:
pd.read_sql_query('select * from Avocado_Import_Data', con=engine).head()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [308]:
pd.read_sql_query('select * from Avocados_by_Region', con=engine).head()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)

In [309]:
pd.read_sql_query('select * from Avocado_Export_Data2', con=engine).head()

OperationalError: (MySQLdb._exceptions.OperationalError) (1045, "Access denied for user '<username>'@'localhost' (using password: YES)") (Background on this error at: http://sqlalche.me/e/e3q8)